In [1]:
import numpy as np

In [2]:
def flip(arr,ind):
    new_arr = arr.copy()
    new_arr[ind] = 1 - arr[ind]
    return new_arr
    
def MCMC(n_chains, iters, n_items, W_i, w):
    # implements random walk in solution space
    x_s = np.empty((iters, n_chains, n_items))
    x_s[0] = np.zeros((n_chains, n_items))
    for i in range(1,iters):
        for c in range(n_chains):
            if 1==np.random.randint(2):
                flip_ind = np.random.randint(n_items)
                x_s[i, c] = flip(x_s[i-1, c],flip_ind)
                if np.dot(x_s[i, c], w) > W_i:
                    x_s[i, c] = flip(x_s[i, c],flip_ind)
            else:
                x_s[i, c] = x_s[i-1, c]
                
    return x_s[int(iters/4*3):] #discard three quaters as warmup

def estimate_n_solutions(items, W):
    running_weight = 0
    weight_last = 0
    iters = 20000
    n_chains = 2
    n_items = len(items)
    omegainv = 1
    for i in range(n_items): #divide problem into subproblems
        running_weight += items[i]
        weight = min(W, running_weight)
        x_s = MCMC(n_chains, iters, n_items, weight, items) #sample from solution space
        n_samples = int(iters/4 * n_chains)
        x_s = x_s.reshape(n_samples, n_items)
        rho = len([1 for x in x_s if np.dot(x,items) <= weight_last])/n_samples
        omegainv *= rho
        weight_last = weight
    return 1/omegainv

In [4]:
item_weights = list(range(1,8))
weight_limit = 13
estimate_n_solutions(item_weights,weight_limit)

53.4631959189924